In [1]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential, load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import joblib

In [ ]:
df = pd.read_csv('final_specs_int_4_class.csv')

In [ ]:
data = []
label =[]

for lab, img in enumerate(df.path):
  try:
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array 
    image = img_to_array(image) 
    # reshape data for the model
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    data.append(image)
    label.append(df.label[lab])
    print(lab, img)
  except:
    print("Error loading image", lab)
  

#Converting lists into numpy arrays
data = np.array(data)
label = np.array(label)

print(data.shape, label.shape)

In [ ]:
X_name = 'X_final_specs_int4_mel.joblib'
y_name = 'y_final_specs_int4_mel.joblib'
save_dir = 'RTSC_joblibs'

savedX = joblib.dump(data, os.path.join(save_dir, X_name))
savedy = joblib.dump(label, os.path.join(save_dir, y_name))

In [2]:
X = joblib.load('RTSC_joblibs/X_final_specs_int4_mel.joblib')
y = joblib.load('RTSC_joblibs/y_final_specs_int4_mel.joblib')

In [ ]:
print(X.shape, y.shape)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

In [24]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7383, 224, 224, 3) (821, 224, 224, 3) (7383,) (821,)


In [ ]:
"""from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, to_categorical
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))
"""

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [4]:
# example of loading the resnet50 model
from keras.applications.resnet50 import ResNet50
# load model
model = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [5]:
for layer in model.layers[:]:
  layer.trainable = False

for layer in model.layers[:]:
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000020D0E229C70> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x0000020D0E264DC0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020D0E279910> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x0000020D0E279160> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000020D2BA61D00> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x0000020D2BA613D0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020D2BA7D220> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020D2BAE1CA0> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x0000020D2BAE72B0> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000020D2BB0EA90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020D2

In [6]:
new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dense( 6 , activation='softmax'))
# summarize the model
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6150      
Total params: 126,355,334
Trainable params: 102,767,622
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:
from keras.optimizers import SGD, RMSprop
opt = RMSprop(lr=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)

In [8]:
new_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
from keras.callbacks import ModelCheckpoint

In [10]:
EPOCHS = 10

checkpoint_filepath = 'Resnet50_int_4_class.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath= checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

new_model.fit(X_train, y_train, batch_size=16, epochs=EPOCHS,validation_data = (X_test, y_test), callbacks=[model_checkpoint_callback])

Epoch 1/10
462/462 [==============================] - 1083s 2s/step - loss: 3.4251 - accuracy: 0.5584 - val_loss: 2.9219 - val_accuracy: 0.5713
Epoch 2/10
462/462 [==============================] - 1076s 2s/step - loss: 1.7221 - accuracy: 0.6767 - val_loss: 1.9927 - val_accuracy: 0.6468
Epoch 3/10
462/462 [==============================] - 1077s 2s/step - loss: 1.2988 - accuracy: 0.7200 - val_loss: 0.9110 - val_accuracy: 0.7479
Epoch 4/10
462/462 [==============================] - 1069s 2s/step - loss: 1.0146 - accuracy: 0.7612 - val_loss: 1.1785 - val_accuracy: 0.7065
Epoch 5/10
462/462 [==============================] - 1071s 2s/step - loss: 0.8595 - accuracy: 0.7963 - val_loss: 1.7654 - val_accuracy: 0.6687
Epoch 6/10
462/462 [==============================] - 1070s 2s/step - loss: 0.7088 - accuracy: 0.8165 - val_loss: 1.0456 - val_accuracy: 0.7381
Epoch 7/10
462/462 [==============================] - 1070s 2s/step - loss: 0.6077 - accuracy: 0.8368 - val_loss: 1.3900 - val_accuracy:

In [ ]:
model_saved = load_model('Resnet50_int_4_class.h5')

In [11]:
predictions = new_model.predict_classes(X_test)
predictions

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([3, 1, 2, 0, 1, 0, 2, 0, 0, 3, 3, 3, 3, 0, 3, 1, 3, 0, 1, 1, 3, 1,
       0, 2, 2, 0, 1, 0, 2, 2, 3, 2, 2, 2, 0, 1, 0, 1, 3, 3, 2, 1, 2, 3,
       2, 2, 0, 2, 3, 2, 0, 0, 0, 0, 1, 1, 0, 3, 1, 2, 2, 3, 1, 2, 2, 2,
       1, 3, 1, 2, 0, 2, 0, 0, 1, 3, 2, 2, 2, 3, 1, 2, 3, 2, 2, 2, 0, 0,
       1, 3, 2, 2, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 0, 0, 3,
       1, 1, 0, 1, 3, 2, 2, 3, 0, 1, 2, 3, 0, 1, 3, 0, 0, 1, 2, 1, 0, 2,
       0, 2, 1, 0, 3, 3, 0, 2, 2, 3, 3, 2, 3, 3, 2, 1, 2, 0, 0, 1, 2, 2,
       2, 2, 0, 3, 1, 0, 3, 2, 1, 2, 0, 3, 3, 2, 2, 2, 2, 0, 1, 0, 1, 3,
       2, 3, 1, 0, 3, 0, 0, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 0, 3, 0, 3, 2,
       0, 3, 0, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 0, 1, 3, 1, 3, 3, 2, 1, 3,
       0, 2, 1, 0, 2, 1, 2, 0, 3, 1, 1, 2, 0, 1, 0, 3, 1, 3, 1, 1, 0, 2,
       1, 1, 0, 2, 3, 3, 2, 2, 2, 0, 0, 2, 1, 2, 3, 0, 3, 2, 2, 2, 3, 2,
       3, 2, 3, 2, 3, 1, 2, 3, 3, 2, 0, 3, 2, 1, 2, 0, 3, 0, 3, 3, 1, 0,
       3, 0, 1, 1, 3, 3, 2, 0, 3, 3, 3, 2, 3, 2, 1,

In [12]:
new_Ytest = y_test.astype(int)
new_Ytest

array([3, 1, 0, 0, 3, 1, 2, 0, 1, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 1, 3, 1,
       0, 2, 2, 0, 3, 0, 2, 2, 1, 2, 0, 0, 0, 1, 0, 1, 3, 3, 0, 1, 2, 3,
       2, 1, 2, 2, 3, 2, 0, 0, 0, 2, 1, 1, 1, 3, 1, 2, 1, 3, 1, 2, 2, 0,
       2, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 0, 0, 1, 1, 3, 2, 2, 0, 0, 0,
       1, 3, 0, 2, 1, 1, 2, 0, 0, 2, 1, 3, 2, 2, 1, 2, 0, 3, 2, 0, 0, 3,
       1, 3, 0, 3, 3, 2, 3, 3, 0, 1, 2, 3, 0, 1, 1, 3, 0, 1, 2, 1, 0, 0,
       0, 0, 3, 0, 3, 1, 0, 2, 2, 3, 3, 2, 3, 3, 2, 1, 2, 0, 0, 1, 0, 2,
       2, 2, 0, 3, 1, 0, 3, 2, 1, 2, 0, 3, 3, 2, 2, 2, 3, 0, 1, 0, 1, 3,
       0, 3, 1, 0, 3, 0, 0, 2, 0, 0, 3, 2, 0, 2, 1, 2, 1, 0, 3, 2, 2, 2,
       1, 3, 2, 1, 2, 1, 2, 2, 1, 2, 1, 3, 3, 1, 0, 3, 3, 3, 3, 0, 3, 1,
       3, 2, 1, 0, 2, 3, 1, 1, 3, 1, 1, 0, 0, 1, 0, 3, 1, 3, 1, 1, 0, 2,
       1, 1, 0, 2, 1, 3, 0, 2, 2, 0, 0, 2, 1, 2, 3, 0, 3, 3, 1, 0, 3, 2,
       3, 2, 1, 0, 3, 1, 2, 3, 3, 2, 0, 3, 2, 1, 1, 0, 3, 0, 3, 3, 3, 0,
       3, 0, 1, 1, 3, 3, 2, 0, 3, 3, 3, 2, 1, 1, 1,

In [13]:
from sklearn.metrics import classification_report
report = classification_report(new_Ytest, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       203
           1       0.78      0.68      0.73       188
           2       0.66      0.86      0.75       207
           3       0.93      0.74      0.83       223

    accuracy                           0.76       821
   macro avg       0.77      0.76      0.76       821
weighted avg       0.78      0.76      0.76       821



In [14]:

loss, acc = new_model.evaluate(X_test, new_Ytest)
print("Model accuracy: {:5.2f}%".format(100*acc))


26/26 [==============================] - 67s 3s/step - loss: 1.1404 - accuracy: 0.7576
Model accuracy: 75.76%


In [ ]:
EPOCHS = 20

checkpoint_filepath = 'Resnet50_6_intclass_1.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath= checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_saved.fit(X_train, y_train, batch_size=16, epochs=EPOCHS,validation_data = (X_test, y_test), callbacks=[model_checkpoint_callback])

In [ ]:
model_saved_1 = load_model('Resnet50_6_intclass_1.h5')

In [ ]:
EPOCHS = 20

checkpoint = 'Resnet50_6_intclass_2.h5'
model_checkpoint_callback = ModelCheckpoint(filepath = checkpoint, save_weights_only = False, monitor ='val_accuracy', mode = 'max', save_best_only = True)

model_saved_1.fit(X_train, y_train, batch_size = 16,epochs = EPOCHS, validation_data = (X_test, y_test), callbacks =[model_checkpoint_callback])

In [15]:
df1 = pd.read_csv('test_final_specs_int_4_class.csv')

In [16]:
data = []
label =[]

for lab, img in enumerate(df1.path):
  try:
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array 
    image = img_to_array(image) 
    # reshape data for the model
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    data.append(image)
    label.append(df1.label[lab])
    print(lab, img)
  except:
    print("Error loading image", lab)
  

#Converting lists into numpy arrays
data = np.array(data)
label = np.array(label)

print(data.shape, label.shape)

0 specs_test\angry\10-52-04-91-56-26-36.png
1 specs_test\angry\10-83-04-43-92-95-87.png
2 specs_test\angry\10-94-04-29-18-33-75.png
3 specs_test\angry\11-22-04-27-66-13-42.png
4 specs_test\angry\12-45-04-16-82-20-15.png
5 specs_test\angry\12-63-04-19-94-73-43.png
6 specs_test\angry\13-24-04-43-86-53-78.png
7 specs_test\angry\13-60-04-82-75-36-14.png
8 specs_test\angry\14-19-04-62-92-23-97.png
9 specs_test\angry\14-87-04-92-36-88-95.png
10 specs_test\angry\15-69-04-62-84-33-37.png
11 specs_test\angry\16-23-04-29-46-18-77.png
12 specs_test\angry\16-65-04-58-97-18-19.png
13 specs_test\angry\16-91-04-63-39-92-25.png
14 specs_test\angry\18-19-04-40-25-31-68.png
15 specs_test\angry\18-20-04-80-78-61-95.png
16 specs_test\angry\18-60-04-47-58-12-37.png
17 specs_test\angry\18-75-04-45-91-56-85.png
18 specs_test\angry\20-43-04-38-96-71-11.png
19 specs_test\angry\21-80-04-14-36-13-51.png
20 specs_test\angry\22-51-04-26-92-14-54.png
21 specs_test\angry\23-37-04-42-51-76-95.png
22 specs_test\angry\

194 specs_test\happy\95-13-02-32-51-67-55.png
195 specs_test\happy\95-51-02-55-28-64-54.png
196 specs_test\happy\96-15-02-22-33-93-10.png
197 specs_test\happy\97-35-02-78-43-42-47.png
198 specs_test\neutral\12-76-01-59-51-68-55.png
199 specs_test\neutral\12-92-01-11-72-69-61.png
200 specs_test\neutral\13-10-01-18-39-53-82.png
201 specs_test\neutral\14-56-01-76-85-80-34.png
202 specs_test\neutral\14-94-01-29-93-15-63.png
203 specs_test\neutral\14-96-01-87-89-44-11.png
204 specs_test\neutral\15-29-01-67-82-38-76.png
205 specs_test\neutral\15-63-01-50-69-10-87.png
206 specs_test\neutral\17-66-01-64-91-18-25.png
207 specs_test\neutral\18-21-01-29-73-47-74.png
208 specs_test\neutral\19-23-01-59-89-83-98.png
209 specs_test\neutral\19-35-01-15-18-25-36.png
210 specs_test\neutral\20-79-01-49-78-54-82.png
211 specs_test\neutral\22-83-01-85-12-69-90.png
212 specs_test\neutral\23-58-01-30-91-19-22.png
213 specs_test\neutral\26-68-01-45-51-75-42.png
214 specs_test\neutral\27-39-01-57-35-41-17.png


In [17]:
X_name = 'X_final_specs_int_4__test.joblib'
y_name = 'y_final_specs_int_4_test.joblib'
save_dir = 'RTSC_joblibs'

savedX = joblib.dump(data, os.path.join(save_dir, X_name))
savedy = joblib.dump(label, os.path.join(save_dir, y_name))

In [19]:
X = joblib.load('RTSC_joblibs/X_final_specs_int_4__test.joblib')
y = joblib.load('RTSC_joblibs/y_final_specs_int_4_test.joblib')

In [20]:
predictions = new_model.predict_classes(X)
predictions

array([3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 3, 3, 1, 3,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1,
       3, 1, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3,
       1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1,
       3, 1, 3, 1, 3, 1, 1, 1, 1, 3, 3, 1, 3, 3, 3, 3, 2, 1, 3, 1, 1, 1,
       3, 3, 1, 2, 3, 3, 3, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3,
       2, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 1, 3, 3, 1, 3, 1, 3,
       2, 2, 0, 0, 2, 2, 2, 0, 2, 3, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0,
       0, 0, 0, 2, 0, 2, 3, 0, 0, 0, 0, 1, 1, 0, 2, 3, 2, 0, 2, 0, 0, 1,
       0, 0, 3, 0, 3, 3, 0, 2, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 2, 0,
       1, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2,

In [21]:
from sklearn.metrics import classification_report
report = classification_report(y, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.56      0.65        79
           1       0.48      0.45      0.46        71
           2       0.62      0.73      0.67        62
           3       0.68      0.77      0.72       127

    accuracy                           0.65       339
   macro avg       0.64      0.63      0.63       339
weighted avg       0.65      0.65      0.64       339



In [22]:

loss, acc = new_model.evaluate(X, y)
print("Model accuracy: {:5.2f}%".format(100*acc))


11/11 [==============================] - 26s 2s/step - loss: 1.6252 - accuracy: 0.6460
Model accuracy: 64.60%


In [23]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y, predictions)
print (matrix)

[[44  4 24  7]
 [ 0 32  3 36]
 [12  2 45  3]
 [ 0 29  0 98]]
